In [1]:
from scipy.stats import spearmanr
import pandas as pd
import numpy as np
from dtaidistance import dtw
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter

In [11]:
# === Paths ===
base_dir = './altis_power_res/no_power_shift/mem_throughput'
baseline_path = f'{base_dir}/max_uncore/bfs.csv'
magus_path = f'{base_dir}/dynamic_uncore/bfs.csv'

# === Read CSVs ===
df_max = pd.read_csv(baseline_path)
df_mag = pd.read_csv(magus_path)


# === Extract throughput column ===
T_max = df_max[' total(MB/s)'].values
T_MAGUS = df_mag[' total(MB/s)'].values

# === Ensure equal length (optional: truncate or interpolate) ===
min_len = min(len(T_max), len(T_MAGUS))
T_max = T_max[:min_len]
T_MAGUS = T_MAGUS[:min_len]

# === 1. Spearman Correlation ===
spearman_corr, _ = spearmanr(T_max, T_MAGUS)
print(f'Spearman correlation: {spearman_corr:.3f}')

Spearman correlation: 0.696


In [2]:
base_dir = './altis_power_res/no_power_shift/mem_throughput'
baseline_path = f'{base_dir}/max_uncore/bfs.csv'
magus_path = f'{base_dir}/dynamic_uncore/bfs.csv'

# === Read CSVs ===
df_max = pd.read_csv(baseline_path)
df_mag = pd.read_csv(magus_path)

# === Extract throughput column ===
T_max = df_max[' total(MB/s)'].values
T_MAGUS = df_mag[' total(MB/s)'].values

# === Ensure equal length ===
min_len = min(len(T_max), len(T_MAGUS))
T_max = T_max[:min_len]
T_MAGUS = T_MAGUS[:min_len]

# === 1. Original Spearman Correlation ===
orig_spearman, _ = spearmanr(T_max, T_MAGUS)
orig_dtw = dtw.distance(T_max, T_MAGUS)
print(f'[Original] Spearman: {orig_spearman:.3f}, DTW similarity: {1/(1+orig_dtw):.3f}')

# === Step 1: Smoothing (Savitzky–Golay) ===
T_max_smooth = savgol_filter(T_max, window_length=7, polyorder=2)
T_MAGUS_smooth = savgol_filter(T_MAGUS, window_length=7, polyorder=2)

# === Step 2: Downsample (optional factor=5) ===
def downsample(series, factor=5):
    series = series[:len(series)//factor * factor]  # trim to be divisible
    return np.mean(series.reshape(-1, factor), axis=1)

T_max_ds = downsample(T_max_smooth, factor=5)
T_MAGUS_ds = downsample(T_MAGUS_smooth, factor=5)

# === Step 3: Z-Normalize (shape comparison only) ===
def zscore(series):
    return (series - np.mean(series)) / np.std(series)

T_max_z = zscore(T_max_ds)
T_MAGUS_z = zscore(T_MAGUS_ds)

# === Step 4: Lag alignment (cross-correlation shift) ===
def best_lag(a, b):
    a_z = zscore(a)
    b_z = zscore(b)
    xcorr = np.correlate(a_z, b_z, mode='full')
    lag = np.argmax(xcorr) - (len(b) - 1)
    return lag

lag = best_lag(T_max_z, T_MAGUS_z)
if lag > 0:
    T_MAGUS_shifted = np.pad(T_MAGUS_z, (lag, 0), mode='edge')[:len(T_MAGUS_z)]
else:
    T_MAGUS_shifted = np.pad(T_MAGUS_z, (0, -lag), mode='edge')[-lag:]

# === Final Spearman and DTW Similarity ===
spearman_final, _ = spearmanr(T_max_z, T_MAGUS_shifted)
dtw_final = dtw.distance(T_max_z, T_MAGUS_shifted)
print(f'[Enhanced] Spearman: {spearman_final:.3f}')

[Original] Spearman: 0.696, DTW similarity: 0.000
[Enhanced] Spearman: 0.821


In [8]:
# === Settings ===
base_dir = './altis_power_res/no_power_shift/mem_throughput'
benchmarks = [
    "bfs", "gemm", "pathfinder", "sort", "cfd", "cfd_double", "fdtd2d",
    "kmeans", "lavamd", "nw", "particlefilter_float", "raytracing", "srad", "where"
]

# === Helper Functions ===
def downsample(series, factor=5):
    series = series[:len(series)//factor * factor]
    return np.mean(series.reshape(-1, factor), axis=1)

def zscore(series):
    return (series - np.mean(series)) / np.std(series)

def best_lag(a, b):
    a_z = zscore(a)
    b_z = zscore(b)
    xcorr = np.correlate(a_z, b_z, mode='full')
    lag = np.argmax(xcorr) - (len(b) - 1)
    return lag

# === Result Collection ===
results = []

for app in benchmarks:
    baseline_path = f'{base_dir}/max_uncore/{app}.csv'
    magus_path = f'{base_dir}/dynamic_uncore/{app}.csv'
    
    try:
        # Read CSVs
        df_max = pd.read_csv(baseline_path)
        df_mag = pd.read_csv(magus_path)

        T_max = df_max[' total(MB/s)'].values
        T_MAGUS = df_mag[' total(MB/s)'].values

        min_len = min(len(T_max), len(T_MAGUS))
        T_max = T_max[:min_len]
        T_MAGUS = T_MAGUS[:min_len]

        # Original Spearman & DTW
        orig_spearman, _ = spearmanr(T_max, T_MAGUS)
        orig_dtw = dtw.distance(T_max, T_MAGUS)
        orig_dtw_sim = 1 / (1 + orig_dtw)

        # === Enhancement Steps ===
        T_max_smooth = savgol_filter(T_max, window_length=7, polyorder=2)
        T_MAGUS_smooth = savgol_filter(T_MAGUS, window_length=7, polyorder=2)

        T_max_ds = downsample(T_max_smooth)
        T_MAGUS_ds = downsample(T_MAGUS_smooth)

        T_max_z = zscore(T_max_ds)
        T_MAGUS_z = zscore(T_MAGUS_ds)

        lag = best_lag(T_max_z, T_MAGUS_z)
        if lag > 0:
            T_MAGUS_shifted = np.pad(T_MAGUS_z, (lag, 0), mode='edge')[:len(T_MAGUS_z)]
        else:
            T_MAGUS_shifted = np.pad(T_MAGUS_z, (0, -lag), mode='edge')[-lag:]

        # Final Spearman & DTW
        spearman_final, _ = spearmanr(T_max_z, T_MAGUS_shifted)
        dtw_final = dtw.distance(T_max_z, T_MAGUS_shifted)
        dtw_final_sim = 1 / (1 + dtw_final)

        results.append({
            "app": app,
            "orig_spearman": round(orig_spearman, 3),
            "orig_dtw_sim": round(orig_dtw_sim, 3),
            "final_spearman": round(spearman_final, 3),
            "final_dtw_sim": round(dtw_final_sim, 3),
        })

    except Exception as e:
        results.append({
            "app": app,
            "error": str(e)
        })

# === Print Results ===
print("\n=== Pattern Similarity Results ===")
for r in results:
    print(f"{r['app']:<25} Spearman: {r['final_spearman']}")


=== Pattern Similarity Results ===
bfs                       Spearman: 0.821
gemm                      Spearman: 1.0
pathfinder                Spearman: 0.288
sort                      Spearman: 0.556
cfd                       Spearman: 0.301
cfd_double                Spearman: 0.756
fdtd2d                    Spearman: 0.732
kmeans                    Spearman: 0.473
lavamd                    Spearman: 0.681
nw                        Spearman: 0.938
particlefilter_float      Spearman: 0.513
raytracing                Spearman: 0.413
srad                      Spearman: 0.614
where                     Spearman: 0.348


In [15]:
# === Settings ===
base_dir = './ecp_power_res/no_power_shift/mem_throughput'

benchmarks = ['Laghos', 'miniGAN', 'sw4lite','UNet','Resnet50','bert_large','lammps','gromacs']
# benchmarks = ['miniGAN']

# === Helper Functions ===
def downsample(series, factor=5):
    series = series[:len(series)//factor * factor]
    return np.mean(series.reshape(-1, factor), axis=1)

def zscore(series):
    return (series - np.mean(series)) / np.std(series)

def best_lag(a, b):
    a_z = zscore(a)
    b_z = zscore(b)
    xcorr = np.correlate(a_z, b_z, mode='full')
    lag = np.argmax(xcorr) - (len(b) - 1)
    return lag

# === Result Collection ===
results = []

for app in benchmarks:
    baseline_path = f'{base_dir}/max_uncore/{app}.csv'
    magus_path = f'{base_dir}/dynamic_uncore/{app}.csv'
    
    
    # Read CSVs
    df_max = pd.read_csv(baseline_path)
    df_mag = pd.read_csv(magus_path)

    T_max = df_max[' total(MB/s)'].values
    T_MAGUS = df_mag[' total(MB/s)'].values

    min_len = min(len(T_max), len(T_MAGUS))
    T_max = T_max[:min_len]
    T_MAGUS = T_MAGUS[:min_len]

    # Original Spearman & DTW
    orig_spearman, _ = spearmanr(T_max, T_MAGUS)
    orig_dtw = dtw.distance(T_max, T_MAGUS)
    orig_dtw_sim = 1 / (1 + orig_dtw)

    # === Enhancement Steps ===
    T_max_smooth = savgol_filter(T_max, window_length=7, polyorder=2)
    T_MAGUS_smooth = savgol_filter(T_MAGUS, window_length=7, polyorder=2)

    T_max_ds = downsample(T_max_smooth)
    T_MAGUS_ds = downsample(T_MAGUS_smooth)

    T_max_z = zscore(T_max_ds)
    T_MAGUS_z = zscore(T_MAGUS_ds)

    lag = best_lag(T_max_z, T_MAGUS_z)
    if lag > 0:
        T_MAGUS_shifted = np.pad(T_MAGUS_z, (lag, 0), mode='edge')[:len(T_MAGUS_z)]
    else:
        T_MAGUS_shifted = np.pad(T_MAGUS_z, (0, -lag), mode='edge')[-lag:]

    # Final Spearman & DTW
    spearman_final, _ = spearmanr(T_max_z, T_MAGUS_shifted)
    dtw_final = dtw.distance(T_max_z, T_MAGUS_shifted)
    dtw_final_sim = 1 / (1 + dtw_final)
    
    results.append({
        "app": app,
        "orig_spearman": round(orig_spearman, 3),
        "orig_dtw_sim": round(orig_dtw_sim, 3),
        "final_spearman": round(spearman_final, 3),
        "final_dtw_sim": round(dtw_final_sim, 3),
    })

# === Print Results ===
print("\n=== Pattern Similarity Results ===")
for r in results:
    print(f"{r['app']:<25} Spearman: {r['final_spearman']}")


=== Pattern Similarity Results ===
Laghos                    Spearman: 0.406
miniGAN                   Spearman: 0.435
sw4lite                   Spearman: 0.158
UNet                      Spearman: 0.433
Resnet50                  Spearman: 0.658
bert_large                Spearman: 0.381
lammps                    Spearman: 0.001
gromacs                   Spearman: 0.349
